In [ ]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

**-----------Load The Data------------**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [ ]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 7 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [ ]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/higgs_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [ ]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, MQ)
]

In [ ]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**--------QuickSelect Algorithm----------------**

In [ ]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

**----------KD Tree---------------------**

In [ ]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous data storage
        if n == 2*self.leaf_size:
            self.leaf_size = n
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

**-------KD Tree Node Size Optimization For Rebuild-------**

**--------------Run 1---------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2_500  # Ensure inclusive range
leaf_size_step = 50

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 392.21it/s]


-------------------------------------------------------------
Build Time: 27.6983 sec, Search Time: 0.7846 sec
Leaf Size: 50, Total Time: 28.4829 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 389.17it/s]


-------------------------------------------------------------
Build Time: 22.9696 sec, Search Time: 0.7772 sec
Leaf Size: 100, Total Time: 23.7469 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 14.8532 sec, Search Time: 0.7203 sec
Leaf Size: 2,400, Total Time: 15.5736 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 420.52it/s]


-------------------------------------------------------------
Build Time: 14.8577 sec, Search Time: 0.7093 sec
Leaf Size: 2,450, Total Time: 15.5670 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 410.42it/s]

-------------------------------------------------------------
Build Time: 14.9134 sec, Search Time: 0.7528 sec
Leaf Size: 2,500, Total Time: 15.6663 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 27.6983 sec, Search Time: 0.7846 sec, Total Time: 28.4829 sec
Leaf Size: 100, Build Time: 22.9696 sec, Search Time: 0.7772 sec, Total Time: 23.7469 sec
Leaf Size: 150, Build Time: 21.6321 sec, Search Time: 0.8011 sec, Total Time: 22.4331 sec
Leaf Size: 200, Build Time: 21.8932 sec, Search Time: 0.7932 sec, Total Time: 22.6864 sec
Leaf Size: 250, Build Time: 20.3979 sec, Search Time: 0.7661 sec, Total Time: 21.1640 sec
Leaf Size: 300, Build Time: 20.4793 sec, Search Time: 0.7481 sec, Total Time: 21.2274 sec
Leaf Size: 350, Build Time: 23.5031 sec, Search Time: 0.7558 sec, Total Time: 24.2588 sec
Leaf Size: 400, Build Time: 24.8889 sec, Search Time: 0.7671 sec, Total Time: 25.6560 sec
Leaf Size: 450, Build Time: 20.3618 sec, Search Time: 0.7714 sec, Total Time: 21.1332

**-------------Run 2-----------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2500  # Ensure inclusive range
leaf_size_step = 50

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 427.63it/s]


-------------------------------------------------------------
Build Time: 22.8468 sec, Search Time: 0.7171 sec
Leaf Size: 50, Total Time: 23.5639 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 391.11it/s]


-------------------------------------------------------------
Build Time: 20.7972 sec, Search Time: 0.7305 sec
Leaf Size: 100, Total Time: 21.5276 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 14.4637 sec, Search Time: 0.6852 sec
Leaf Size: 2,400, Total Time: 15.1489 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 435.49it/s]


-------------------------------------------------------------
Build Time: 14.3619 sec, Search Time: 0.6921 sec
Leaf Size: 2,450, Total Time: 15.0540 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 402.35it/s]

-------------------------------------------------------------
Build Time: 14.5566 sec, Search Time: 0.7050 sec
Leaf Size: 2,500, Total Time: 15.2615 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 22.8468 sec, Search Time: 0.7171 sec, Total Time: 23.5639 sec
Leaf Size: 100, Build Time: 20.7972 sec, Search Time: 0.7305 sec, Total Time: 21.5276 sec
Leaf Size: 150, Build Time: 19.7849 sec, Search Time: 0.7113 sec, Total Time: 20.4962 sec
Leaf Size: 200, Build Time: 19.5356 sec, Search Time: 0.7194 sec, Total Time: 20.2549 sec
Leaf Size: 250, Build Time: 18.0569 sec, Search Time: 0.6968 sec, Total Time: 18.7537 sec
Leaf Size: 300, Build Time: 17.9780 sec, Search Time: 0.6657 sec, Total Time: 18.6437 sec
Leaf Size: 350, Build Time: 17.7657 sec, Search Time: 0.6937 sec, Total Time: 18.4593 sec
Leaf Size: 400, Build Time: 17.7044 sec, Search Time: 0.6763 sec, Total Time: 18.3808 sec
Leaf Size: 450, Build Time: 17.0551 sec, Search Time: 0.6926 sec, Total Time: 17.7478

**-------------Opt Node Size For Rebuild-------------**

In [ ]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array([  50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700,
  750, 800, 850, 900, 950,1000,1050,1100,1150,1200,1250,1300,1350,1400,
 1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,2050,2100,
 2150,2200,2250,2300,2350,2400,2450,2500])

# Total times from two runs (or versions)
total_times_1 = np.array([28.4829,23.7469,22.4331,22.6864,21.164 ,21.2274,24.2588,25.656 ,21.1332,
 20.0302,20.4376,19.6157,19.538 ,19.5222,19.3926,19.6623,19.5609,19.2034,
 18.1255,17.2173,16.7467,18.3682,17.7691,18.1305,17.4116,16.845 ,16.7859,
 16.4995,16.7799,16.9242,16.9046,16.8496,16.7376,16.906 ,16.2161,15.9482,
 15.9873,15.3897,15.3793,15.3343,15.3893,15.4401,15.388 ,15.3618,15.4765,
 14.9935,15.3795,15.5736,15.567 ,15.6663])

total_times_2 = np.array([23.5639,21.5276,20.4962,20.2549,18.7537,18.6437,18.4593,18.3808,17.7478,
 17.4671,17.286 ,17.5021,17.5503,17.6158,17.2617,17.1613,17.0677,16.7443,
 16.7762,16.2921,16.0885,16.115 ,16.1555,16.3493,16.1702,16.3343,16.1179,
 15.9319,16.1783,15.9753,15.9654,16.0153,15.9772,16.0289,15.6254,15.4549,
 15.2255,15.4137,15.0781,15.1108,15.0214,15.2545,15.1946,15.2693,15.1585,
 15.0333,14.9428,15.1489,15.054 ,15.2615])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 15.0134
Minimum found at leaf size: 2300


--------------------------------------------------

**-----Epsilon Optimization For Rebuild------**

**-----------Run 1-------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size = 2300
# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.1
epsilon_size_step = 0.001

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 521.71it/s]


-------------------------------------------------------------
Build Time: 12.1909 sec, Search Time: 0.5174 sec
Epsilon: 0.0, Total Time: 12.7083 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.001): 100%|██████████| 100/100 [00:00<00:00, 579.03it/s]


-------------------------------------------------------------
Build Time: 12.4283 sec, Search Time: 0.4933 sec
Epsilon: 0.001, Total Time: 12.9216 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 8.6515 sec, Search Time: 1.3805 sec
Epsilon: 0.098, Total Time: 10.0320 sec

100: === Testing with Epsilon: 0.099, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.099): 100%|██████████| 100/100 [00:01<00:00, 88.45it/s]


-------------------------------------------------------------
Build Time: 8.7100 sec, Search Time: 1.3659 sec
Epsilon: 0.099, Total Time: 10.0759 sec

101: === Testing with Epsilon: 0.1, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.1): 100%|██████████| 100/100 [00:01<00:00, 85.07it/s]

-------------------------------------------------------------
Build Time: 8.7851 sec, Search Time: 1.4038 sec
Epsilon: 0.1, Total Time: 10.1889 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 12.1909 sec, Search Time: 0.5174 sec, Total Time: 12.7083 sec
Epsilon: 0.001, Build Time: 12.4283 sec, Search Time: 0.4933 sec, Total Time: 12.9216 sec
Epsilon: 0.002, Build Time: 12.4237 sec, Search Time: 0.4996 sec, Total Time: 12.9234 sec
Epsilon: 0.003, Build Time: 12.4363 sec, Search Time: 0.5231 sec, Total Time: 12.9594 sec
Epsilon: 0.004, Build Time: 12.5792 sec, Search Time: 0.5159 sec, Total Time: 13.0952 sec
Epsilon: 0.005, Build Time: 12.3877 sec, Search Time: 0.5425 sec, Total Time: 12.9302 sec
Epsilon: 0.006, Build Time: 12.3292 sec, Search Time: 0.5483 sec, Total Time: 12.8775 sec
Epsilon: 0.007, Build Time: 12.4283 sec, Search Time: 0.5722 sec, Total Time: 13.0005 sec
Epsilon: 0.008, Build Time: 12.1929 sec, Search Time: 0.5815 sec, Total Time: 12.7744 se

**----------Run 2--------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size = 2300
# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.1
epsilon_size_step = 0.001

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 517.85it/s]


-------------------------------------------------------------
Build Time: 12.5788 sec, Search Time: 0.5208 sec
Epsilon: 0.0, Total Time: 13.0996 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.001): 100%|██████████| 100/100 [00:00<00:00, 572.36it/s]


-------------------------------------------------------------
Build Time: 12.8567 sec, Search Time: 0.5008 sec
Epsilon: 0.001, Total Time: 13.3575 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 8.9403 sec, Search Time: 1.3568 sec
Epsilon: 0.098, Total Time: 10.2971 sec

100: === Testing with Epsilon: 0.099, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.099): 100%|██████████| 100/100 [00:01<00:00, 88.45it/s]


-------------------------------------------------------------
Build Time: 9.0012 sec, Search Time: 1.3634 sec
Epsilon: 0.099, Total Time: 10.3646 sec

101: === Testing with Epsilon: 0.1, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.1): 100%|██████████| 100/100 [00:01<00:00, 86.24it/s]

-------------------------------------------------------------
Build Time: 8.9210 sec, Search Time: 1.3964 sec
Epsilon: 0.1, Total Time: 10.3174 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 12.5788 sec, Search Time: 0.5208 sec, Total Time: 13.0996 sec
Epsilon: 0.001, Build Time: 12.8567 sec, Search Time: 0.5008 sec, Total Time: 13.3575 sec
Epsilon: 0.002, Build Time: 12.4387 sec, Search Time: 0.5168 sec, Total Time: 12.9555 sec
Epsilon: 0.003, Build Time: 12.5793 sec, Search Time: 0.5239 sec, Total Time: 13.1032 sec
Epsilon: 0.004, Build Time: 12.5175 sec, Search Time: 0.5301 sec, Total Time: 13.0476 sec
Epsilon: 0.005, Build Time: 12.2545 sec, Search Time: 0.5653 sec, Total Time: 12.8198 sec
Epsilon: 0.006, Build Time: 12.5116 sec, Search Time: 0.5569 sec, Total Time: 13.0685 sec
Epsilon: 0.007, Build Time: 12.4095 sec, Search Time: 0.5622 sec, Total Time: 12.9718 sec
Epsilon: 0.008, Build Time: 12.4200 sec, Search Time: 0.5762 sec, Total Time: 12.9962 se

**--------Epsilon Opt Value For Rebuild------------**

In [ ]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array([0.   ,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01 ,0.011,
 0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.02 ,0.021,0.022,0.023,
 0.024,0.025,0.026,0.027,0.028,0.029,0.03 ,0.031,0.032,0.033,0.034,0.035,
 0.036,0.037,0.038,0.039,0.04 ,0.041,0.042,0.043,0.044,0.045,0.046,0.047,
 0.048,0.049,0.05 ,0.051,0.052,0.053,0.054,0.055,0.056,0.057,0.058,0.059,
 0.06 ,0.061,0.062,0.063,0.064,0.065,0.066,0.067,0.068,0.069,0.07 ,0.071,
 0.072,0.073,0.074,0.075,0.076,0.077,0.078,0.079,0.08 ,0.081,0.082,0.083,
 0.084,0.085,0.086,0.087,0.088,0.089,0.09 ,0.091,0.092,0.093,0.094,0.095,
 0.096,0.097,0.098,0.099,0.1  ])

# Total times from two runs (or versions)
total_times_1 = np.array([12.7083,12.9216,12.9234,12.9594,13.0952,12.9302,12.8775,13.0005,12.7744,
 13.0025,12.6864,12.7121,12.8017,12.3593,12.645 ,12.604 ,12.5455,12.4747,
 12.33  ,12.4602,12.187 ,12.3864,12.4352,12.1914,12.1221,12.1639,11.9282,
 12.1604,12.0092,12.0352,11.9825,11.7786,11.9576,12.0467,12.053 ,11.9006,
 11.9719,11.8617,11.7868,12.1098,11.9651,11.981 ,11.8502,12.0957,11.7755,
 11.7246,11.5972,11.6415,11.5637,11.5887,11.4644,11.4544,11.2394,11.4367,
 11.4065,11.2276,11.3103,11.3059,11.1022,11.2901,11.1948,11.0463,10.9451,
 10.9253,10.8954,10.9361,10.8183,10.8155,10.7288,10.6667,10.9691,10.8713,
 10.671 ,10.61  ,10.7133,10.7072,10.6811,10.4905,10.3958,10.5004,10.3872,
 10.6512,10.3872,10.5384,10.454 ,10.3489,10.5122,10.3957,10.5426,10.248 ,
 10.4538,10.236 ,10.1829,10.2312,10.3369,10.2909,10.1836,10.1686,10.032 ,
 10.0759,10.1889])

total_times_2 = np.array([13.0996,13.3575,12.9555,13.1032,13.0476,12.8198,13.0685,12.9718,12.9962,
 12.9041,13.1206,12.8805,12.832 ,12.5459,12.7384,12.6894,12.4527,12.3941,
 12.8997,12.4804,12.682 ,12.5748,12.5161,12.4797,12.636 ,12.3883,12.3947,
 12.3387,12.3603,12.3207,12.349 ,12.2138,12.3346,12.1173,12.1374,12.1551,
 12.3463,12.2028,12.0417,12.4058,12.2112,12.3314,12.3535,12.2813,11.8555,
 11.9473,12.033 ,11.7524,11.7717,11.5771,11.6968,11.7859,11.5349,11.4591,
 11.4652,11.323 ,11.4604,11.3409,11.3708,11.2566,11.105 ,11.1458,10.9496,
 11.0031,11.1996,11.0656,10.9568,10.7929,10.8419,10.9935,10.8972,10.9246,
 10.8649,10.7343,10.619 ,10.6132,10.8488,10.6787,10.5703,10.5892,10.7923,
 10.5503,10.7295,10.5768,10.6591,10.5102,10.5266,10.5463,10.5026,10.5477,
 10.5114,10.3545,10.4888,10.6417,10.4432,10.4102,10.4055,10.4026,10.2971,
 10.3646,10.3174])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 10.1646
Minimum found at epsilon size: 0.098


-------------------

**------------Epsilon Optimization for Insertion-------------**

**-------------------Run 1---------------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2300
epsilon_rebuild = 0.098

# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.1
epsilon_size_step = 0.001

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Readjust for the insertion
    data_size, query_size = test_combinations[0]
    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    kdtree.epsilon = epsilon

    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 151.36it/s]


-------------------------------------------------------------
Build Time: 8.6739 sec, Search Time: 1.3803 sec
Epsilon: 0.0, Total Time: 10.0542 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.001): 100%|██████████| 100/100 [00:00<00:00, 152.64it/s]


-------------------------------------------------------------
Build Time: 9.0463 sec, Search Time: 1.3517 sec
Epsilon: 0.001, Total Time: 10.3980 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 8.3526 sec, Search Time: 1.7453 sec
Epsilon: 0.098, Total Time: 10.0979 sec

100: === Testing with Epsilon: 0.099, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.099): 100%|██████████| 100/100 [00:01<00:00, 99.28it/s]


-------------------------------------------------------------
Build Time: 8.2703 sec, Search Time: 1.7426 sec
Epsilon: 0.099, Total Time: 10.0129 sec

101: === Testing with Epsilon: 0.1, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.1): 100%|██████████| 100/100 [00:01<00:00, 98.67it/s]

-------------------------------------------------------------
Build Time: 8.4158 sec, Search Time: 1.7374 sec
Epsilon: 0.1, Total Time: 10.1532 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.6739 sec, Search Time: 1.3803 sec, Total Time: 10.0542 sec
Epsilon: 0.001, Build Time: 9.0463 sec, Search Time: 1.3517 sec, Total Time: 10.3980 sec
Epsilon: 0.002, Build Time: 8.8424 sec, Search Time: 1.3560 sec, Total Time: 10.1984 sec
Epsilon: 0.003, Build Time: 9.0234 sec, Search Time: 1.3531 sec, Total Time: 10.3765 sec
Epsilon: 0.004, Build Time: 8.8212 sec, Search Time: 1.3519 sec, Total Time: 10.1731 sec
Epsilon: 0.005, Build Time: 8.6605 sec, Search Time: 1.3091 sec, Total Time: 9.9695 sec
Epsilon: 0.006, Build Time: 8.6163 sec, Search Time: 1.3582 sec, Total Time: 9.9745 sec
Epsilon: 0.007, Build Time: 8.9437 sec, Search Time: 1.3912 sec, Total Time: 10.3348 sec
Epsilon: 0.008, Build Time: 8.8269 sec, Search Time: 1.3871 sec, Total Time: 10.2141 sec
Epsilon: 

**----------------Run 2-------------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2300
epsilon_rebuild = 0.098

# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.1
epsilon_size_step = 0.001

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Readjust for the insertion
    data_size, query_size = test_combinations[0]
    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    kdtree.epsilon = epsilon

    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 149.82it/s]


-------------------------------------------------------------
Build Time: 8.8663 sec, Search Time: 1.3978 sec
Epsilon: 0.0, Total Time: 10.2641 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.001): 100%|██████████| 100/100 [00:00<00:00, 145.54it/s]


-------------------------------------------------------------
Build Time: 8.8167 sec, Search Time: 1.4106 sec
Epsilon: 0.001, Total Time: 10.2273 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 8.4333 sec, Search Time: 1.9300 sec
Epsilon: 0.098, Total Time: 10.3633 sec

100: === Testing with Epsilon: 0.099, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.099): 100%|██████████| 100/100 [00:01<00:00, 88.72it/s]


-------------------------------------------------------------
Build Time: 8.2607 sec, Search Time: 1.9498 sec
Epsilon: 0.099, Total Time: 10.2104 sec

101: === Testing with Epsilon: 0.1, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.1): 100%|██████████| 100/100 [00:01<00:00, 98.72it/s]

-------------------------------------------------------------
Build Time: 8.5436 sec, Search Time: 1.7869 sec
Epsilon: 0.1, Total Time: 10.3305 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.8663 sec, Search Time: 1.3978 sec, Total Time: 10.2641 sec
Epsilon: 0.001, Build Time: 8.8167 sec, Search Time: 1.4106 sec, Total Time: 10.2273 sec
Epsilon: 0.002, Build Time: 8.8115 sec, Search Time: 1.4020 sec, Total Time: 10.2135 sec
Epsilon: 0.003, Build Time: 8.9142 sec, Search Time: 1.4510 sec, Total Time: 10.3652 sec
Epsilon: 0.004, Build Time: 8.7685 sec, Search Time: 1.4235 sec, Total Time: 10.1920 sec
Epsilon: 0.005, Build Time: 8.6927 sec, Search Time: 1.4317 sec, Total Time: 10.1244 sec
Epsilon: 0.006, Build Time: 8.7536 sec, Search Time: 1.4599 sec, Total Time: 10.2135 sec
Epsilon: 0.007, Build Time: 8.7624 sec, Search Time: 1.4671 sec, Total Time: 10.2295 sec
Epsilon: 0.008, Build Time: 8.9211 sec, Search Time: 1.4806 sec, Total Time: 10.4017 sec
Epsilon

**------Epsilon Optimization For Insertion----**

In [ ]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array([0.   ,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01 ,0.011,
 0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.02 ,0.021,0.022,0.023,
 0.024,0.025,0.026,0.027,0.028,0.029,0.03 ,0.031,0.032,0.033,0.034,0.035,
 0.036,0.037,0.038,0.039,0.04 ,0.041,0.042,0.043,0.044,0.045,0.046,0.047,
 0.048,0.049,0.05 ,0.051,0.052,0.053,0.054,0.055,0.056,0.057,0.058,0.059,
 0.06 ,0.061,0.062,0.063,0.064,0.065,0.066,0.067,0.068,0.069,0.07 ,0.071,
 0.072,0.073,0.074,0.075,0.076,0.077,0.078,0.079,0.08 ,0.081,0.082,0.083,
 0.084,0.085,0.086,0.087,0.088,0.089,0.09 ,0.091,0.092,0.093,0.094,0.095,
 0.096,0.097,0.098,0.099,0.1  ])

# Total times from two runs (or versions)
total_times_1 = np.array([10.2641,10.2273,10.2135,10.3652,10.192 ,10.1244,10.2135,10.2295,10.4017,
 10.4498,10.3385,10.3463,10.3562,10.2766,10.3254,10.0192,10.1761,10.2419,
 10.1845,10.319 ,10.3751,10.2761,10.265 ,10.1436,10.2409,10.216 ,10.2263,
 10.2264,10.1617,10.2431,10.1775,10.0749,10.351 ,10.3085,10.2155,10.0869,
 10.1091,10.3084,10.3472,10.4393,10.5197,10.2251,10.2478,10.2197,10.2118,
 10.1606, 9.911 ,10.0521,10.031 ,10.1028,10.3454,10.0104,10.1334,10.1543,
 10.183 ,10.183 ,10.0102,10.032 , 9.8909, 9.9196,10.0284,10.2685,10.0901,
 10.2615,10.0514,10.0963,10.1067,10.2898,10.0916,10.0287,10.0355,10.1382,
 10.1309,10.158 , 9.984 ,10.0567, 9.8996,10.1022, 9.9354,10.1345,10.0813,
 10.101 ,10.0219,10.375 ,10.0879,10.0391,10.0493,10.0566,10.2988,10.0923,
 10.2229,10.2405,10.2534,10.1364,10.1677, 9.9638,10.1579,10.6407,10.3633,
 10.2104,10.3305])

total_times_2 = np.array( [10.0542,10.398 ,10.1984,10.3765,10.1731, 9.9695, 9.9745,10.3348,10.2141,
 10.2965,10.596 ,10.1227,10.0615,10.3521,10.3114,10.6117,10.4288,10.2917,
 10.3314,10.3184,10.2892,10.3282,10.3351,10.1945,10.4573,10.2858,10.1757,
 10.4193,10.1248,10.3909,10.5513,10.3681,10.444 ,10.2271,10.2183,10.2659,
 10.2748,10.7182,10.1916,10.224 ,10.2835,10.0652,10.3167, 9.9405,10.1393,
  9.9419,10.1583,10.1813,10.0763,10.2501,10.0329,10.4029,10.2138,10.1596,
 10.1686,10.1892,10.3054,10.3639,10.1947,10.1379,10.1388,10.1223,10.1619,
 10.1088, 9.9235,10.1106,10.1039,10.1177, 9.8334,10.2544,10.0437,10.2115,
 10.2115,10.1053,10.3701,10.0794,10.0759,10.0291,10.1921,10.0205,10.0786,
 10.1239,10.2818,10.3234, 9.8564,10.0279,10.0535,10.1593, 9.8547,10.0116,
 10.0372,10.1344, 9.8673,10.2195,10.1551, 9.8944, 9.8719, 9.8902,10.0979,
 10.0129,10.1532])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 9.9291
Minimum found at epsilon size: 0.095
